In [2]:
import arcticdb as adb
import dotenv
import pandas as pd
from eodhd import APIClient
import os

dotenv.load_dotenv(dotenv_path="../.env")


In [48]:
ac = adb.Arctic('lmdb://../data_storage/database')
# ac.create_library('data')  # fixed schema - see note below
ac.list_libraries()
library = ac['data']

In [3]:
api_key = os.environ.get("EODHD_API_KEY")
api = APIClient(api_key)

In [7]:
exchange_df = api.get_exchanges()

In [65]:
tradeable_exchanges = [
    'NASDAQ',
    'NYSE',
    'NYSE MKT',
    'NYSE ARCA',
    'NYSEARCA',
    'BATS',
    'AMEX'
]

In [61]:
symbols = api.get_exchange_symbols(uri='US')

In [71]:
active_stocks = symbols[symbols['Exchange'].isin(tradeable_exchanges) & (symbols['Type'] == 'Common Stock')]

In [91]:
active_stocks['Code'][0:100]

4          A
5         AA
53      AACG
56      AACI
58     AACIW
       ...  
710      ADC
717     ADCT
721      ADD
728     ADEA
733     ADEX
Name: Code, Length: 100, dtype: object

In [94]:
for code in active_stocks['Code'][0:10].tolist():
    test = api.get_fundamentals_data(ticker=f'{code}.US')

In [62]:
delisted_symbols = api.get_exchange_symbols(uri='US', delisted=True)

In [72]:
delisted_stocks = delisted_symbols[delisted_symbols['Exchange'].isin(tradeable_exchanges) & (delisted_symbols['Type'] == 'Common Stock')]

In [74]:
delisted_stocks

,Code,Name,Country,Exchange,Currency,Type,Isin
0,AAAB,Admiralty Bancorp Inc,USA,NASDAQ,USD,Common Stock,None
2,AAAGY,Altana Aktiengesellschaft,USA,NYSE,USD,Common Stock,None
6,AAAP,Advanced Accelerator Applications S.A,USA,NASDAQ,USD,Common Stock,US00790T1007
9,AAB,Astrazeneca Ab,USA,NYSE,USD,Common Stock,None
10,AABA,Altaba Inc,USA,NASDAQ,USD,Common Stock,US0213461017
...,...,...,...,...,...,...,...
47489,ZX,China Zenix Auto International Limited,USA,NYSE,USD,Common Stock,US16951E1047
47491,ZXYZ-A,NASDAQ SYMBOLOGY TEST,USA,NASDAQ,USD,Common Stock,None
47492,ZY,Zymergen Inc,USA,NASDAQ,USD,Common Stock,US98985X1000
47494,ZYNE,Zynerba Pharmaceuticals Inc,USA,NASDAQ,USD,Common Stock,US98986X1090


In [87]:
fundamental_data = api.get_fundamentals_data(ticker='ABCM.US')

In [88]:
income_statement_df = pd.DataFrame(fundamental_data['Financials']['Income_Statement']['quarterly']).transpose()

In [26]:
income_statement_df = (income_statement_df
                       .reset_index(drop=True)
                       .astype()
                       )

In [50]:
date_fields = ['date', 'filing_date']
string_field = ['currency_symbol']
float_field_mask = ~income_statement_df.columns.isin(date_fields + string_field)
float_column_names = income_statement_df.columns[float_field_mask]

In [53]:
income_statement_df[date_fields] = income_statement_df[date_fields].apply(pd.to_datetime)
income_statement_df[string_field] = income_statement_df[string_field].astype(str)
income_statement_df[float_column_names] = income_statement_df[float_column_names].astype(float)

In [54]:
income_statement_df

,date,filing_date,currency_symbol,researchDevelopment,effectOfAccountingCharges,incomeBeforeTax,minorityInterest,netIncome,sellingGeneralAdministrative,sellingAndMarketingExpenses,...,otherItems,incomeTaxExpense,totalRevenue,totalOperatingExpenses,costOfRevenue,totalOtherIncomeExpenseNet,discontinuedOperations,netIncomeFromContinuingOps,netIncomeApplicableToCommonShares,preferredStockAndOtherAdjustments
0,2024-03-31,2024-05-03,USD,7903000000.00000000,NaN,28058000000.00000000,NaN,23636000000.00000000,6468000000.00000000,NaN,...,NaN,4422000000.00000000,90753000000.00000000,14371000000.00000000,48482000000.00000000,158000000.00000000,NaN,23636000000.00000000,NaN,NaN
1,2023-12-31,2024-02-02,USD,7696000000.00000000,NaN,40323000000.00000000,NaN,33916000000.00000000,6786000000.00000000,NaN,...,NaN,6407000000.00000000,119575000000.00000000,14482000000.00000000,64720000000.00000000,-50000000.00000000,NaN,33916000000.00000000,NaN,NaN
2,2023-09-30,2023-11-03,USD,7307000000.00000000,NaN,26998000000.00000000,NaN,22956000000.00000000,6151000000.00000000,NaN,...,NaN,4042000000.00000000,89498000000.00000000,13458000000.00000000,49071000000.00000000,29000000.00000000,NaN,22956000000.00000000,22956000000.00000000,NaN
3,2023-06-30,2023-08-04,USD,7442000000.00000000,NaN,22733000000.00000000,NaN,19881000000.00000000,5973000000.00000000,NaN,...,NaN,2852000000.00000000,81797000000.00000000,13415000000.00000000,45384000000.00000000,-265000000.00000000,NaN,19881000000.00000000,19881000000.00000000,NaN
4,2023-03-31,2023-05-05,USD,7457000000.00000000,NaN,28382000000.00000000,NaN,24160000000.00000000,6201000000.00000000,NaN,...,NaN,4222000000.00000000,94836000000.00000000,13658000000.00000000,52860000000.00000000,64000000.00000000,NaN,24160000000.00000000,24160000000.00000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,1986-09-30,1986-09-30,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,510800000.00000000,-1628400000.00000000,NaN,NaN,NaN,NaN,NaN,NaN
151,1986-06-30,1986-06-30,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,448300000.00000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
152,1986-03-31,1986-03-31,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,408900000.00000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153,1985-12-31,1985-12-31,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,533900000.00000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
library.write('income_statement', income_statement_df)

VersionedItem(symbol='income_statement', library='data', data=n/a, version=0, metadata=None, host='LMDB(path=C:\\Users\\mason\\PycharmProjects\\quantitative-finance\\data_storage\\database)', timestamp=1721181452715667400)

In [56]:
from_storage_df = library.read('income_statement').data

In [57]:
from_storage_df

,date,filing_date,currency_symbol,researchDevelopment,effectOfAccountingCharges,incomeBeforeTax,minorityInterest,netIncome,sellingGeneralAdministrative,sellingAndMarketingExpenses,...,otherItems,incomeTaxExpense,totalRevenue,totalOperatingExpenses,costOfRevenue,totalOtherIncomeExpenseNet,discontinuedOperations,netIncomeFromContinuingOps,netIncomeApplicableToCommonShares,preferredStockAndOtherAdjustments
0,2024-03-31,2024-05-03,USD,7903000000.00000000,NaN,28058000000.00000000,NaN,23636000000.00000000,6468000000.00000000,NaN,...,NaN,4422000000.00000000,90753000000.00000000,14371000000.00000000,48482000000.00000000,158000000.00000000,NaN,23636000000.00000000,NaN,NaN
1,2023-12-31,2024-02-02,USD,7696000000.00000000,NaN,40323000000.00000000,NaN,33916000000.00000000,6786000000.00000000,NaN,...,NaN,6407000000.00000000,119575000000.00000000,14482000000.00000000,64720000000.00000000,-50000000.00000000,NaN,33916000000.00000000,NaN,NaN
2,2023-09-30,2023-11-03,USD,7307000000.00000000,NaN,26998000000.00000000,NaN,22956000000.00000000,6151000000.00000000,NaN,...,NaN,4042000000.00000000,89498000000.00000000,13458000000.00000000,49071000000.00000000,29000000.00000000,NaN,22956000000.00000000,22956000000.00000000,NaN
3,2023-06-30,2023-08-04,USD,7442000000.00000000,NaN,22733000000.00000000,NaN,19881000000.00000000,5973000000.00000000,NaN,...,NaN,2852000000.00000000,81797000000.00000000,13415000000.00000000,45384000000.00000000,-265000000.00000000,NaN,19881000000.00000000,19881000000.00000000,NaN
4,2023-03-31,2023-05-05,USD,7457000000.00000000,NaN,28382000000.00000000,NaN,24160000000.00000000,6201000000.00000000,NaN,...,NaN,4222000000.00000000,94836000000.00000000,13658000000.00000000,52860000000.00000000,64000000.00000000,NaN,24160000000.00000000,24160000000.00000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,1986-09-30,1986-09-30,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,510800000.00000000,-1628400000.00000000,NaN,NaN,NaN,NaN,NaN,NaN
151,1986-06-30,1986-06-30,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,448300000.00000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
152,1986-03-31,1986-03-31,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,408900000.00000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153,1985-12-31,1985-12-31,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,533900000.00000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
